In [1]:
import pandas as pd 
import numpy as np 
import matplotlib as px
import seaborn as sns
import plotly as plt 
from sklearn.metrics.pairwise import haversine_distances
import math 
import haversine
import pygeodesic

# 0.1 Loading Data

In [2]:
df = pd.read_csv('./data/train.crdownload.csv')

## 1.0 DATA DESCRIPTIONS

In [3]:
df1 = df.copy()

In [4]:
df1.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


## 1.1 Data Dimensions

In [5]:
print(f'Number of Rows {df1.shape[0]}')
print(f'Number of Columns {df1.shape[1]}')

Number of Rows 4467
Number of Columns 20


In [6]:
df1.sample(1)

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
1855,0x1893,MUMRES17DEL03,33,3.6,19.121999,72.908493,19.211999,72.998493,29-03-2022,19:25:00,19:30:00,conditions Windy,Jam,1,Drinks,motorcycle,1,No,Metropolitian,(min) 34


In [7]:
df1.dtypes

ID                              object
Delivery_person_ID              object
Delivery_person_Age             object
Delivery_person_Ratings         object
Restaurant_latitude            float64
Restaurant_longitude           float64
Delivery_location_latitude     float64
Delivery_location_longitude    float64
Order_Date                      object
Time_Orderd                     object
Time_Order_picked               object
Weatherconditions               object
Road_traffic_density            object
Vehicle_condition                int64
Type_of_order                   object
Type_of_vehicle                 object
multiple_deliveries             object
Festival                        object
City                            object
Time_taken(min)                 object
dtype: object

In [8]:
df1.isnull().sum()

ID                             0
Delivery_person_ID             0
Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Order_Date                     0
Time_Orderd                    0
Time_Order_picked              0
Weatherconditions              0
Road_traffic_density           0
Vehicle_condition              0
Type_of_order                  0
Type_of_vehicle                0
multiple_deliveries            0
Festival                       0
City                           1
Time_taken(min)                1
dtype: int64

## 1.3 Change Types

In [16]:
# Delivery_person_Age
df1['Delivery_person_Age'] = df1['Delivery_person_Age'].str.strip()
df1['Delivery_person_Age'] = df1['Delivery_person_Age'].replace('[^\d]+', '', regex=True)
df1['Delivery_person_Age'] = df1['Delivery_person_Age'].replace('', np.nan)
df1['Delivery_person_Age'] = df1['Delivery_person_Age'].astype(float)

AttributeError: Can only use .str accessor with string values!

In [17]:
df1.dtypes

ID                              object
Delivery_person_ID              object
Delivery_person_Age            float64
Delivery_person_Ratings         object
Restaurant_latitude            float64
Restaurant_longitude           float64
Delivery_location_latitude     float64
Delivery_location_longitude    float64
Order_Date                      object
Time_Orderd                     object
Time_Order_picked               object
Weatherconditions               object
Road_traffic_density            object
Vehicle_condition                int64
Type_of_order                   object
Type_of_vehicle                 object
multiple_deliveries             object
Festival                        object
City                            object
Time_taken(min)                 object
dtype: object

In [18]:
# Delivery_person_Age
df1['Delivery_person_Ratings'] = df1['Delivery_person_Ratings'].astype('float64')

# Delivery_person_Age
df1['Delivery_person_Ratings'] = df1['Delivery_person_Ratings'].astype('float64')

# Order_Date

df1['Order_Date'] = pd.to_datetime(df1['Order_Date'], errors='coerce')
df1['Order_Date'] = df1['Order_Date'].dt.time

# # Time_taken 
# df1['Time_taken'] = pd.to_datetime(df1['Time_taken'], format='%H:%M:%S').dt.time

# multiple_deliveries

df1['multiple_deliveries'] = df1['multiple_deliveries'].astype('float64')

C:\Users\rozen\AppData\Local\Temp\ipykernel_2612\1235182504.py:9: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df1['Order_Date'] = pd.to_datetime(df1['Order_Date'], errors='coerce')
